### Sequence
동물 분류

In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

root = './datasets/animals/original/'

idg = ImageDataGenerator(rescale=1./255)

generator = idg.flow_from_directory(root, target_size=(64, 64), batch_size=32, class_mode='categorical')
print(generator.class_indices)

Found 26179 images belonging to 10 classes.
{'butterfly': 0, 'cat': 1, 'chicken': 2, 'cow': 3, 'dog': 4, 'elephant': 5, 'horse': 6, 'sheep': 7, 'spider': 8, 'squirrel': 9}


In [3]:
target_name = {v: k for k, v in generator.class_indices.items()}
target_name

{0: 'butterfly',
 1: 'cat',
 2: 'chicken',
 3: 'cow',
 4: 'dog',
 5: 'elephant',
 6: 'horse',
 7: 'sheep',
 8: 'spider',
 9: 'squirrel'}

In [4]:
target_names = []
for target in generator.classes:
    target_names.append(target_name[target])

In [5]:
import pandas as pd

animal_df = pd.DataFrame({'file_paths': generator.filepaths, 'target_names': target_names, 'targets': generator.classes})
animal_df.file_paths = animal_df.file_paths.apply(lambda file_path: file_path.replace('\\', '/'))
animal_df

,file_paths,target_names,targets
0,./datasets/animals/original/butterfly/butterfl...,butterfly,0
1,./datasets/animals/original/butterfly/butterfl...,butterfly,0
2,./datasets/animals/original/butterfly/butterfl...,butterfly,0
3,./datasets/animals/original/butterfly/butterfl...,butterfly,0
4,./datasets/animals/original/butterfly/butterfl...,butterfly,0
...,...,...,...
26174,./datasets/animals/original/squirrel/squirrel9...,squirrel,9
26175,./datasets/animals/original/squirrel/squirrel9...,squirrel,9
26176,./datasets/animals/original/squirrel/squirrel9...,squirrel,9
26177,./datasets/animals/original/squirrel/squirrel9...,squirrel,9


In [6]:
from sklearn.model_selection import train_test_split

train_images, test_images, train_targets, test_targets = \
train_test_split(animal_df.file_paths, 
                 animal_df.targets, 
                 stratify=animal_df.targets, 
                 test_size=0.2, random_state=124)

print(train_targets.value_counts())
print(test_targets.value_counts())

targets
4    3890
8    3857
2    2478
6    2098
0    1690
3    1493
9    1490
7    1456
1    1334
5    1157
Name: count, dtype: int64
targets
4    973
8    964
2    620
6    525
0    422
3    373
9    372
7    364
1    334
5    289
Name: count, dtype: int64


In [7]:
from sklearn.model_selection import train_test_split

train_images, validation_images, train_targets, validation_targets = \
train_test_split(train_images, 
                 train_targets, 
                 stratify=train_targets, 
                 test_size=0.2, random_state=124)

print(train_targets.value_counts())
print(validation_targets.value_counts())
print(test_targets.value_counts())

targets
4    3112
8    3086
2    1982
6    1678
0    1352
3    1194
9    1192
7    1165
1    1067
5     926
Name: count, dtype: int64
targets
4    778
8    771
2    496
6    420
0    338
3    299
9    298
7    291
1    267
5    231
Name: count, dtype: int64
targets
4    973
8    964
2    620
6    525
0    422
3    373
9    372
7    364
1    334
5    289
Name: count, dtype: int64


In [8]:
train_df = animal_df.iloc[train_images.index].reset_index(drop=True)
validation_df = animal_df.iloc[validation_images.index].reset_index(drop=True)
test_df = animal_df.iloc[test_images.index].reset_index(drop=True)

print(train_df.shape)
print(validation_df.shape)
print(test_df.shape)

(16754, 3)
(4189, 3)
(5236, 3)


시퀀스 api는 상속 받아서 내가 재저응 ㅣ
데이터세트 객체를

#### Step1

In [11]:
import numpy as np
from tensorflow.keras.utils import Sequence
from sklearn.utils import shuffle
import cv2

IMAGE_SIZE = 64
BATCH_SIZE = 64

class Dataset(Sequence):
    # 어그먼 테이션 객체를 전달
    def __init__(self, file_paths, targets, batch_size=BATCH_SIZE, aug=None, shuffle=False):
        self.file_paths = file_paths
        self.targets = targets
        self.batch_size = batch_size
        self.aug = aug
        self.shuffle = shuffle

        if self.shuffle:
            # 에포크 종료 시, 객체 생성 및 데이터 섞기
            # 이거 하면 메모리 이슈가 발생함 그래서 직접 해제해야 함
            # 에포크 하나 끝나자마자 실행되는 함수
            self.on_epoch_end()

# 한번 에포크당
# 한번새로우 ㄴ데이터 세트가 만들어짐

# 생성자에서 셔플에서 이프분에서 새로운 객체 만들어짐

    # __len__()는 전체 데이터 건수에서 batch_size 단위로 나눈 데이터 수
    # 하나의 배치 사이즈에 필요한 데이터 개수를 리턴해줘야함
    # 하나의 데이터 세트에 필요한 데이터 개수
    def __len__(self):
        # 예를 들어, 1000개의 데이터를 30 batch_size로 설정하면, 1 batch 당 33.33...개이다.
        # 이때, 소수점은 무조건 올려서 33 + 1 = 34개로 설정한다. (ceil 사용하여 올리기)
        return int(np.ceil(len(self.targets) / self.batch_size))

    # batch_size 단위로 이미지 배열과 타겟 데이터들을 가져온 뒤 변환한 값을 리턴한다. (preprocessing)
    # index: 몇번째 배치인지 인덱스로 나타냄
    def __getitem__(self, index):
        # 시작, 끝 인덱스를 정확하게 가져올 수 있음
        # 배치사이즈 60개라 가정 60:120하면 120은 안가져옴
        file_paths_batch = self.file_paths[index * self.batch_size: (index + 1) * self.batch_size]
        targets_batch = self.targets[index * self.batch_size: (index + 1) * self.batch_size]

        # 배치사이즈, 위드, 하이트, 채널 (튜플로 만들어줘야댐)
        results_batch = np.zeros((file_paths_batch.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3))

        # 데이터 수만큼 반복
        for i in range(file_paths_batch.shape[0]):
            # 이미지 가져오기
            image = cv2.cvtColor(cv2.imread(file_paths_batch[i]), cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))

            # 어그먼테이션 있으면 어그먼 테이션 적용
            if self.aug is not None:
                self.sug(image=image)['image']

            results_batch[i] = image

        return results_batch, targets_batch

    def on_epoch_end(self):
        if self.shuffle:
            # 이걸 먼저 하고 getitem을 하기 때뭉네 result로 안 넣고 file_paths로 넣음
            # 그럼 get_items에는 이미 셔플이 되어있음
            self.file_paths, self.targets = shuffle(self.file_paths, self.targets)

In [15]:
import albumentations as A

train_file_paths = train_df['file_paths'].values
train_targets = train_df['targets'].values

aug = A.Compose([
    A.ShiftScaleRotate(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0, p=0.5)
])

dataset = Dataset(train_file_paths, train_targets, batch_size=BATCH_SIZE, aug=aug, shuffle=True)


#### Step2

In [ ]:
import numpy as np
from tensorflow.keras.utils import Sequence
from sklearn.utils import shuffle
import cv2

IMAGE_SIZE = 64
BATCH_SIZE = 64

class Dataset(Sequence):
    # 어그먼 테이션 객체를 전달
    def __init__(self, file_paths, targets, batch_size=BATCH_SIZE, aug=None, preprocess=None, shuffle=False):
        self.file_paths = file_paths
        self.targets = targets
        self.batch_size = batch_size
        self.aug = aug
        self.preprocess = preprocess
        self.shuffle = shuffle

        if self.shuffle:
            # 에포크 종료 시, 객체 생성 및 데이터 섞기
            # 이거 하면 메모리 이슈가 발생함 그래서 직접 해제해야 함
            # 에포크 하나 끝나자마자 실행되는 함수
            self.on_epoch_end()

# 한번 에포크당
# 한번새로우 ㄴ데이터 세트가 만들어짐

# 생성자에서 셔플에서 이프분에서 새로운 객체 만들어짐

    # __len__()는 전체 데이터 건수에서 batch_size 단위로 나눈 데이터 수
    # 하나의 배치 사이즈에 필요한 데이터 개수를 리턴해줘야함
    # 하나의 데이터 세트에 필요한 데이터 개수
    def __len__(self):
        # 예를 들어, 1000개의 데이터를 30 batch_size로 설정하면, 1 batch 당 33.33...개이다.
        # 이때, 소수점은 무조건 올려서 33 + 1 = 34개로 설정한다. (ceil 사용하여 올리기)
        return int(np.ceil(len(self.targets) / self.batch_size))

    # batch_size 단위로 이미지 배열과 타겟 데이터들을 가져온 뒤 변환한 값을 리턴한다. (preprocessing)
    # index: 몇번째 배치인지 인덱스로 나타냄
    def __getitem__(self, index):
        # 시작, 끝 인덱스를 정확하게 가져올 수 있음
        # 배치사이즈 60개라 가정 60:120하면 120은 안가져옴
        file_paths_batch = self.file_paths[index * self.batch_size: (index + 1) * self.batch_size]
        targets_batch = self.targets[index * self.batch_size: (index + 1) * self.batch_size]

        # 배치사이즈, 위드, 하이트, 채널 (튜플로 만들어줘야댐)
        results_batch = np.zeros((file_paths_batch.shape[0], IMAGE_SIZE, IMAGE_SIZE, 3))

        # 데이터 수만큼 반복
        for i in range(file_paths_batch.shape[0]):
            # 이미지 가져오기
            image = cv2.cvtColor(cv2.imread(file_paths_batch[i]), cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (IMAGE_SIZE, IMAGE_SIZE))

            # 어그먼테이션 있으면 어그먼 테이션 적용
            if self.aug is not None:
                self.aug(image=image)['image']

            if self.preprocess is not None:
                self.self.preprocess(image)

            results_batch[i] = image

        return results_batch, targets_batch

    def on_epoch_end(self):
        if self.shuffle:
            # 이걸 먼저 하고 getitem을 하기 때뭉네 result로 안 넣고 file_paths로 넣음
            # 그럼 get_items에는 이미 셔플이 되어있음
            self.file_paths, self.targets = shuffle(self.file_paths, self.targets)

In [ ]:
import albumentations as A
# 별칭을 줄 수 있음... tf랑 torch랑 모두 같은 input이니까 써줘서 구분 해주기
from tensorflow.keras.applications.xception import 

train_file_paths = train_df['file_paths'].values
train_targets = train_df['targets'].values

validation_file_paths = validation_df['file_paths'].values
validation_targets = validation_df['targets'].values

test_file_paths = test_df['file_paths'].values
test_targets = test_df['targets'].values

aug = A.Compose([
    A.ShiftScaleRotate(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0, p=0.5)
])

train_dataset = Dataset(train_file_paths, train_targets, batch_size=BATCH_SIZE, aug=aug, preprocess=xception_preprocess_input, shuffle=True)



In [ ]:
create_model
모바일넷 작은 장치에서 성능을 끌어올릴 수 있는 모델...